# Test pipeline for processing PDFs and storing in ChromaDB

In [1]:
%load_ext autoreload
%autoreload 2


import glob
import json
import os
import textwrap
from typing import List, Dict, Any
import sys

# Get the absolute path of the current notebook
current_dir = os.getcwd()

# Get the parent directory (which is the 'GenAI' folder)
parent_dir = os.path.dirname(current_dir)

# Add the parent directory to sys.path
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Verify it worked
print(f"Added to path: {parent_dir}")

import chromadb
from tqdm import tqdm

from embeddingModels.BaseEmbeddingModel import BaseEmbeddingModel
from embeddingModels.ModernBertEmbedder import ModernBertEmbedder
from embeddingModels.QwenEmbedder import QwenEmbedder
from pdfProcessing.doclingTest import setup_docling_converter, extract_sections_from_doc, extract_metadata

Added to path: c:\Users\kronask\OneDrive - TU Wien\TU Wien\3. Semester\GenAI\GenAI


In [2]:
import torch

# 1. Check if CUDA (NVIDIA GPU support) is available
gpu_available = torch.cuda.is_available()
print(f"Is GPU available? {gpu_available}")

# 2. If available, print the name of the GPU
if gpu_available:
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")
else:
    print("Running on CPU only.")

Is GPU available? True
GPU Name: NVIDIA GeForce RTX 2080 Ti
CUDA Version: 12.4


## Set up folders

In [3]:
CURRENT_MODEL = "bert"  # Select either qwen or bert
INPUT_FOLDER = "../data/testPDFs"
OUTPUT_FOLDER = "../data/testPDFOutput/pipelineTest"
CHROMA_DB_DIR = "./chroma_db"

In [4]:
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
pdf_files = glob.glob(os.path.join(INPUT_FOLDER, "*.pdf"))
collection_names = {"bert": "scientific_papers_bert", "qwen": "scientific_papers_qwen"}

## Set up ChromaDB Client, Collection and Document Converter

In [5]:
chroma_client = chromadb.PersistentClient(path=CHROMA_DB_DIR)
pipeline_test_collection = chroma_client.get_or_create_collection(
    name=collection_names[CURRENT_MODEL],
    metadata={"hnsw:space": "ip"}
)
converter = setup_docling_converter()
# I used docling from IBM, can also describe images

2026-01-05 20:13:23,703 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


✅ CUDA detected. Using GPU.


### Docling
https://www.docling.ai/

## Convert PDFs and store in json

In [6]:
%%time
for pdf_path in tqdm(pdf_files):
    file_stem = os.path.splitext(os.path.basename(pdf_path))[0]
    try:
        result = converter.convert(pdf_path)

        sections = extract_sections_from_doc(result.document)

        metadata = extract_metadata(sections)

        final_output = {
            "filename": os.path.basename(pdf_path),
            "metadata": metadata,
            "sections": sections
        }

        out_path = os.path.join(OUTPUT_FOLDER, f"{file_stem}_converted.json")
        with open(out_path, "w", encoding="utf-8") as f:
            json.dump(final_output, f, indent=2)

        print(f"✅ Processed: {file_stem}")
        print(f"   found ID: {metadata.get('arxiv_id')}")
        print(f"   found {len(metadata.get('authors', []))} authors")

    except Exception as e:
        print(f"❌ Failed {file_stem}: {e}")


  0%|          | 0/14 [00:00<?, ?it/s]2026-01-05 20:13:24,252 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2026-01-05 20:13:24,287 - INFO - Going to convert document batch...
2026-01-05 20:13:24,289 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 1064fff70b16649e2a9cc84da931292b
2026-01-05 20:13:24,307 - INFO - Loading plugin 'docling_defaults'
2026-01-05 20:13:24,311 - INFO - Registered picture descriptions: ['vlm', 'api']
2026-01-05 20:13:24,325 - INFO - Loading plugin 'docling_defaults'
2026-01-05 20:13:24,334 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2026-01-05 20:13:24,670 - INFO - Accelerator device: 'cuda:0'
[INFO] 2026-01-05 20:13:24,695 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2026-01-05 20:13:24,704 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\kronask\.conda\envs\genai_env\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 202

✅ Processed: Kandel et al. - 2023 - Demonstration of an AI-driven workflow for autonomous high-resolution scanning microscopy
   found ID: None
   found 1 authors


2026-01-05 20:13:51,583 - INFO - Finished converting document Kuprikov et al. - 2022 - Deep reinforcement learning for self-tuning laser source of dissipative solitons.pdf in 13.20 sec.
 14%|█▍        | 2/14 [00:27<02:43, 13.59s/it]2026-01-05 20:13:51,596 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2026-01-05 20:13:51,604 - INFO - Going to convert document batch...
2026-01-05 20:13:51,605 - INFO - Processing document MacLeod et al. - 2022 - A self-driving laboratory advances the Pareto front for material properties.pdf


✅ Processed: Kuprikov et al. - 2022 - Deep reinforcement learning for self-tuning laser source of dissipative solitons
   found ID: None
   found 25 authors


2026-01-05 20:14:00,483 - INFO - Finished converting document MacLeod et al. - 2022 - A self-driving laboratory advances the Pareto front for material properties.pdf in 8.89 sec.
 21%|██▏       | 3/14 [00:36<02:05, 11.45s/it]2026-01-05 20:14:00,493 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2026-01-05 20:14:00,497 - INFO - Going to convert document batch...
2026-01-05 20:14:00,497 - INFO - Processing document Mareev et al. - 2023 - Self-Adjusting Optical Systems Based on Reinforcement Learning.pdf


...

✅ Processed: Volk and Abolhasani - 2024 - Performance metrics to unleash the power of self-driving labs in chemistry and materials science
   found ID: None
   found 3 authors


2026-01-05 20:19:23,759 - INFO - Finished converting document Xie et al. - 2023 - Inverse design of chiral functional films by a robotic AI-guided system.pdf in 22.67 sec.
 93%|█████████▎| 13/14 [05:59<00:36, 36.03s/it]2026-01-05 20:19:23,771 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2026-01-05 20:19:23,785 - INFO - Going to convert document batch...
2026-01-05 20:19:23,787 - INFO - Processing document Zhang et al. - 2024 - Precision autofocus in optical microscopy with liquid lenses controlled by deep reinforcement learni.pdf


✅ Processed: Xie et al. - 2023 - Inverse design of chiral functional films by a robotic AI-guided system
   found ID: None
   found 1 authors


2026-01-05 20:19:38,070 - INFO - Finished converting document Zhang et al. - 2024 - Precision autofocus in optical microscopy with liquid lenses controlled by deep reinforcement learni.pdf in 14.30 sec.
100%|██████████| 14/14 [06:13<00:00, 26.70s/it]

✅ Processed: Zhang et al. - 2024 - Precision autofocus in optical microscopy with liquid lenses controlled by deep reinforcement learni
   found ID: None
   found 1 authors
CPU times: total: 27min 46s
Wall time: 6min 13s


In [7]:
del converter

## Embed and store in ChromaDB

In [8]:
def ingest_papers_to_chroma(
        json_folder: str,
        collection: chromadb.Collection,
        embedding_model: BaseEmbeddingModel
):
    """
    Reads structured JSON papers and ingests them into ChromaDB.
    """

    json_files = glob.glob(os.path.join(json_folder, "*.json"))
    print(f"Found {len(json_files)} JSON files to ingest.")

    for json_file in tqdm(json_files, desc="Processing Papers"):
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # --- A. Determine Parent ID ---
        # Prefer arXiv ID, fallback to filename if missing
        parent_id = data['metadata'].get('arxiv_id')
        if not parent_id:
            parent_id = data['filename']
            # TODO: get ID from sematic scholar

        # Clean ID (Chroma requires IDs to be strings, usually safe chars)
        parent_id = parent_id.replace(" ", "_").replace(":", "_")

        # --- B. Prepare Batches for this Document ---
        documents: List[str] = []
        metadatas: List[Dict[str, Any]] = []
        ids: List[str] = []

        global_meta = {
            "parent_id": parent_id,
            "filename": data['filename'],
            "title": data['metadata'].get('title', "Unknown"),
            "authors": ", ".join(data['metadata'].get('authors', [])),
            "arxiv_id": data['metadata'].get('arxiv_id', "N/A")
        }

        for section_header, content in tqdm(data['sections'].items(), desc="Processing Sections"):
            if not content.strip():
                continue

            # 1. Create Unique ID for this chunk
            safe_header = section_header.replace(" ", "_")[:50]
            chunk_id = f"{parent_id}#{safe_header}"

            # 2. Create Metadata for this chunk
            chunk_meta = global_meta.copy()
            chunk_meta["section"] = section_header
            chunk_meta["is_preamble"] = (section_header == "Preamble")

            # removing \n from content
            content = content.replace("\n", " ")
            documents.append(content)
            metadatas.append(chunk_meta)
            ids.append(chunk_id)

        # --- D. Generate Embeddings ---
        if documents:
            # Use your custom class to encode
            embeddings_np = embedding_model.encode(documents)
            # Convert numpy to python list for Chroma
            embeddings_list = embeddings_np.tolist()

            # --- E. Upsert to Chroma ---
            # using upsert handles re-runs gracefully (updates existing IDs)
            print('Generating embeddings')
            collection.upsert(
                ids=ids,
                embeddings=embeddings_list,
                documents=documents,
                metadatas=metadatas
            )

    print("Ingestion Complete.")

In [9]:
match CURRENT_MODEL:
    case "bert":
        embedder = ModernBertEmbedder(
            model_name="Alibaba-NLP/gte-modernbert-base",
            normalize=True
        )
    case "qwen":
        embedder = QwenEmbedder("Qwen/Qwen3-Embedding-8B", use_fp16=True)


Loading Alibaba-NLP/gte-modernbert-base on cuda...


In [10]:
%%time
ingest_papers_to_chroma(
    json_folder=OUTPUT_FOLDER,
    collection=pipeline_test_collection,
    embedding_model=embedder
)

Found 14 JSON files to ingest.


Processing Sections: 100%|██████████| 21/21 [00:00<?, ?it/s]
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Processing Papers:   7%|▋         | 1/14 [00:02<00:29,  2.23s/it]

Generating embeddings


Processing Papers:  14%|█▍        | 2/14 [00:03<00:17,  1.50s/it]

Generating embeddings


...

Processing Papers:  71%|███████▏  | 10/14 [00:24<00:09,  2.31s/it]

Generating embeddings


Processing Papers:  71%|███████▏  | 10/14 [05:28<02:11, 32.85s/it]


CPU times: total: 5min 29s
Wall time: 5min 28s


OutOfMemoryError: CUDA out of memory. Tried to allocate 48.11 GiB. GPU 0 has a total capacity of 11.00 GiB of which 8.96 GiB is free. Of the allocated memory 766.01 MiB is allocated by PyTorch, and 115.99 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
print(f"Collection Count: {pipeline_test_collection.count()}")

Collection Count: 0


In [ ]:
def query_chroma(
        collection: chromadb.Collection,
        query_text: str,
        model: BaseEmbeddingModel,
        n_results: int = 5,
):
    print(f"--- 🔍 Querying for: '{query_text}' ---")

    try:
        query_vector_np = model.encode([query_text])
        query_vector_list = query_vector_np.tolist()

        results = collection.query(
            query_embeddings=query_vector_list,
            n_results=n_results,
            # Optional: Filter by metadata (e.g., only from specific paper)
            # where={"parent_id": "arXiv:1706.03762v7"}
        )

        # 5. Display Results
        if not results['ids'][0]:
            print("No results found.")
            return

        print(f"\n✅ Found {len(results['ids'][0])} relevant chunks:\n")

        for i in range(len(results['ids'][0])):
            doc_id = results['ids'][0][i]
            score = results['distances'][0][i]  # Lower is better (L2 distance)
            content = results['documents'][0][i]
            metadata = results['metadatas'][0][i]

            print(f"Result #{i + 1} (Distance: {score:.4f})")
            print(f"📄 Paper: {metadata.get('title', 'Unknown')}")
            print(f"📌 Section: {metadata.get('section', 'Unknown')}")
            print(f"🔗 ID: {doc_id}")
            print("-" * 40)
            print("📝 Content Snippet:")
            print(textwrap.fill(content[:300] + "...", width=80))  # Preview first 300 chars
            print("\n" + "=" * 60 + "\n")

        return results
    except Exception as e:
        print(e)

In [ ]:
results = query_chroma(
    collection=pipeline_test_collection,
    query_text="transformer architecture details",
    model=embedder,
    n_results=3
)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


--- 🔍 Querying for: 'transformer architecture details' ---
No results found.
